## 캐싱(Caching)

LangChain은 LLM을 위한 선택적 캐싱 레이어를 제공합니다.

이는 두 가지 이유로 유용합니다.

- 동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 **API 호출 횟수를 줄여 비용을 절감**할 수 있습니다.
- LLM 제공업체에 대한 **API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수** 있습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models-02-Cache-Gemini")

LangSmith 추적을 시작합니다.
[프로젝트명]
project_name='CH04-Models-02-Cache-Gemini'
LANGSMITH_PROJECT: CH04-Models-02-Cache-Gemini


모델과 프롬프트를 생성합니다


In [3]:
from langchain_core.prompts import PromptTemplate
from common.gemini_llm_factory import GeminiLLMFactory

# 모델을 생성합니다.
llm_factory = GeminiLLMFactory()
llm = llm_factory.get_llm()

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm

In [4]:
%%time
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 한반도 남부에 위치한 역동적인 국가입니다. 오랜 역사와 전통을 지녔으며, 한국전쟁의 폐허를 딛고 '한강의 기적'을 이루어 세계적인 경제 대국으로 성장했습니다.

IT 강국이자 혁신 기술을 선도하며, K-팝, K-드라마 등 한류 문화를 통해 전 세계에 영향력을 확대하고 있습니다. 빠르게 변화하며 미래를 선도하는 역동적인 대한민국의 모습은 많은 이들에게 영감을 주고 있습니다.
CPU times: user 49.3 ms, sys: 17 ms, total: 66.3 ms
Wall time: 7.1 s


## InMemoryCache

인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환합니다.

In [5]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 역동적인 국가입니다. 오랜 역사와 전통을 지니고 있으며, 6.25 전쟁 이후 눈부신 경제 성장을 이루어 '한강의 기적'을 일구었습니다.

현재는 IT 기술 강국이자 세계적인 문화 콘텐츠(K-팝, 드라마 등)를 선도하는 나라로 자리매김했습니다. 전통과 현대가 조화롭게 공존하며, 끊임없이 변화하고 발전하는 매력적인 나라입니다.
CPU times: user 413 ms, sys: 22.5 ms, total: 435 ms
Wall time: 6.32 s


In [6]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 역동적인 국가입니다. 오랜 역사와 전통을 지니고 있으며, 6.25 전쟁 이후 눈부신 경제 성장을 이루어 '한강의 기적'을 일구었습니다.

현재는 IT 기술 강국이자 세계적인 문화 콘텐츠(K-팝, 드라마 등)를 선도하는 나라로 자리매김했습니다. 전통과 현대가 조화롭게 공존하며, 끊임없이 변화하고 발전하는 매력적인 나라입니다.
CPU times: user 2.75 ms, sys: 90 μs, total: 2.84 ms
Wall time: 2.61 ms


## SQLite Cache


In [7]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [8]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 반도 국가로, 남북으로 나뉘어 있습니다. 대한민국(남한)은 짧은 역사에도 불구하고 '한강의 기적'을 통해 경제 대국으로 성장했습니다.

IT 강국이자 세계적인 문화 콘텐츠(K-POP, 드라마 등 한류)를 선도하며 전 세계에 영향력을 미치고 있습니다. 역동적인 민주주의와 전통이 현대와 조화롭게 어우러진 독특한 매력을 지닌 나라입니다.
CPU times: user 35.7 ms, sys: 2.68 ms, total: 38.4 ms
Wall time: 6.93 s
